In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
from plotly.offline import init_notebook_mode
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.html.widgets.interaction import interact
%matplotlib inline
init_notebook_mode()

In [ ]:
# path to csv file downloaded from Deutsche bank
filename = "./Transactions.csv"

In [ ]:
cols = ["Value date", "Transaction Type", "Beneficiary / Originator", "Payment Details", "IBAN", "Customer Reference", "Debit", "Credit"]
df = pd.read_csv(filename, 
                 skiprows=4,
                 sep=";", 
                 encoding='latin1', 
                 usecols=cols, 
                 index_col="Value date",
                )
df.index = pd.to_datetime(df.index, format='%m/%d/%Y')

# parse old balance in the beginning
def parse_old_balance(filename):
    with open(filename, 'r') as f:
        next(f);next(f);
        line = f.readline()
        return line.split(";")[-2]
old_balance = parse_old_balance(filename)
old_balance_df = pd.DataFrame({'Transaction Type': 'Old Balance', 'Credit': old_balance}, index=[df.index[0]])
df = pd.concat([old_balance_df, df], sort=True)
df.head(10)

In [ ]:
df['Credit'] = df['Credit'].apply(lambda x: float(str(x).replace(',', '')))
df['Debit'] = df['Debit'].apply(lambda x: float(str(x).replace(',', ''))).abs()
expense_df = df[pd.isna(df['Credit'])].dropna(subset=['Debit'])
income_df = df[pd.isna(df['Debit'])].dropna(subset=['Credit'])

In [ ]:
# make sure that the output of this cell is same as in the
# total balance shown in your bank account (discarding 'pending transactions')
income_df['Credit'].sum() - expense_df['Debit'].sum()

# Monthly Income vs Expense report

## This plot shows your incomes vs expenses for each month

In [ ]:
income_per_month = df["Credit"].resample("M", kind="period").sum()
expense_per_month = df["Debit"].resample("M", kind="period").sum()

data = [go.Bar(x=income_per_month.index.to_timestamp(), y=income_per_month.values, name='income'), 
        go.Bar(x=expense_per_month.index.to_timestamp(), y=expense_per_month.values, name='expense')]
py.iplot(data)

## This plot shows your income vs expense summed over throughout the months. 
You can get an idea of how much you have earned and spent till the given month in total. For e.g. if you are looking at
the values in October, then it shows how much you have earned/spent till the end of October from the beginning

In [ ]:
income_per_month_cumsum = income_per_month.cumsum()
expense_per_month_cum_sum = expense_per_month.cumsum()
data = [go.Bar(x=income_per_month_cumsum.index.to_timestamp(), y=income_per_month_cumsum.values, name='income'), 
        go.Bar(x=expense_per_month_cum_sum.index.to_timestamp(), y=expense_per_month_cum_sum.values, name='expense')]
py.iplot(data)

## Income vs expense where savings from previous month is accumulated

In [ ]:
starting_balance_for_next_month = (income_per_month_cumsum - expense_per_month_cum_sum).shift(1).fillna(0)
added_income_per_month = starting_balance_for_next_month + income_per_month

data = [go.Bar(x=added_income_per_month.index.to_timestamp(), y=added_income_per_month.values, name="income with savings"),
        go.Scatter(x=income_per_month.index.to_timestamp(), y=income_per_month.values, name='income'),
        go.Bar(x=expense_per_month.index.to_timestamp(), y=expense_per_month.values, name='expense')]
py.iplot(data)

# Top income sources

In [ ]:
# pd.Grouper(freq='M') for grouping by index column
top_income_sources = income_df.groupby(by=['Beneficiary / Originator'])['Credit'].sum()
data = [go.Pie(labels=top_income_sources.index, values=top_income_sources.values)]
py.iplot(data)

# Top expense sources
Where do you spend the most?

In [ ]:
import re

# key is the expense's name and value is a list of regex
expense_sources_regexes = {
    'groceries': ['penny', 'rewe', 'aldi', 'lidl', 'netto'],
    'shopping': ['amazon', 'karstadt', 'galleria', 'rossmann'],
    'paypal': ['paypal'],
    'insurance': ['krankenkasse'],
    'phone': ['telefonica', 'drillisch'],
    'cash withdrawal': ['cash\s+withdrawal'],
    'rent': ['studierendenwerk\s+darmstadt']
}

def parse_expense_source(text):
    for k, regexes in expense_sources_regexes.items():
        for r in regexes:
            if re.findall(r, text, re.I):
                return k
    return None


transaction_texts = expense_df['Beneficiary / Originator'].fillna(" ") + " " + expense_df['Payment Details'].fillna(" ") + " " + expense_df['Transaction Type'].fillna(" ")
expense_df['Expense Source'] = transaction_texts.map(parse_expense_source)
# if expense source could not be identified as known expense source using `parse_expense_source`
# then use value from `Beneficiary / Originator` column as the expense source
# if that is also empty then put the text 'other' as expense source
expense_df['Expense Source'] = expense_df['Expense Source'].fillna(expense_df['Beneficiary / Originator']).fillna("other")

In [ ]:
def plot_expense_report(month):
    if month == 'All':
        top_expense_sources = expense_df.groupby(by=['Expense Source'])['Debit'].sum()
    else:
        top_expense_sources = expense_df[month].groupby(by=['Expense Source'])['Debit'].sum()
    
    top_expense_sources = top_expense_sources.sort_values(ascending=False)[:15]
    data = [go.Pie(labels=top_expense_sources.index, values=top_expense_sources.values)]
    py.iplot(data)

labels = ['All'] + list(expense_df.index.to_period(freq='M').unique().map(str))
month_widget = widgets.Dropdown(options=dict(zip(labels, labels)), value=labels[0])

_ = interact(plot_expense_report, month=month_widget)

# Expense breakdown by expense source
Compare how much you spend per 'expense source (rent, groceries etc.)' for a given range

In [ ]:
def plot_comparison(expense, **kwargs):
    print(expense, kwargs)
    if not expense:
        return
    date1, date2 = kwargs['from_date'], kwargs['till_date']
    
    try:
        filtered = expense_df[expense_df['Expense Source'] == expense][date1: date2]['Debit'].resample("M", kind="period").sum()
        bar = go.Scatter(x=filtered.index.to_timestamp(), y=filtered.values, name=expense, )

        py.iplot([bar])
    except (KeyError, AttributeError) as e:
        print("No data found for ", expense, date1, date2)

labels = expense_df['Expense Source'].unique()
labels.sort()
expenses_widget = widgets.Dropdown(options=dict(zip(labels, labels)))

labels = expense_df.index.to_period(freq='M').unique().map(str)
date1_widget = widgets.Dropdown(options=dict(zip(labels, labels)), value=labels[0])
date2_widget = widgets.Dropdown(options=dict(zip(labels[1:], labels[1:])), value = labels[-1])

_ = interact(plot_comparison, expense=expenses_widget, from_date=date1_widget, till_date=date2_widget)